In [50]:
import paddle
import paddle.nn.functional as F
import numpy as np
import pandas as pd
import os
from paddle import nn
from PIL import Image
from paddle.distributed import fleet, get_rank
from visualdl import LogWriter

IMAGE_SIZE = 1440
BATCH_SIZE = 32
EPOCH_NUM = 5

print(paddle.__version__)

2.4.1


In [51]:
# 定义数据集
class MyDataset(paddle.io.Dataset):
    def __init__(self, img_dir='data/PALM-Training400/', csv_dir='data/Classification.csv') -> None:
        super(MyDataset, self).__init__()
        if csv_dir is None:
            self.csvfile = None
            self.filedir = os.listdir(img_dir)
        else:
            self.csvfile = pd.read_csv(csv_dir)
        self.imgpath = img_dir
        pass
    def __len__(self):
        if self.csvfile is None:
            return len(self.filedir)
        else:
            return len(self.csvfile)
        pass
    def __getitem__(self, idx):
        if self.csvfile is None:
            img = np.reshape((np.array(Image.open(self.imgpath+os.sep+self.filedir[idx]).resize((IMAGE_SIZE,IMAGE_SIZE))).astype('float32')),(3,IMAGE_SIZE,IMAGE_SIZE))/256.
            lab = self.filedir[idx]
        else:
            img = np.reshape((np.array(Image.open(self.imgpath+os.sep+self.csvfile['imgName'][idx]).resize((IMAGE_SIZE,IMAGE_SIZE))).astype('float32')),(3,IMAGE_SIZE,IMAGE_SIZE))/256.
            lab = np.array(self.csvfile['Label'][idx]).astype('float32')
        return img,lab
    pass
mydataset = MyDataset()

In [52]:
x = paddle.reshape(paddle.to_tensor(mydataset[0][0]),(-1,3,IMAGE_SIZE,IMAGE_SIZE))
x

Tensor(shape=[1, 3, 1440, 1440], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [[[[0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          ...,
          [0.        , 0.        , 0.        , ..., 0.51562500,
           0.33984375, 0.19921875],
          [0.51171875, 0.34765625, 0.20312500, ..., 0.37890625,
           0.21093750, 0.11328125],
          [0.37500000, 0.20703125, 0.11328125, ..., 0.        ,
           0.        , 0.        ]],

         [[0.        , 0.        , 0.        , ..., 0.51562500,
           0.34375000, 0.19531250],
          [0.51171875, 0.35156250, 0.19921875, ..., 0.37890625,
           0.19921875, 0.11328125],
          [0.37890625, 0.20312500, 0.11328125, ..., 0.        ,
           0.        , 0.      

In [81]:
net = nn.Sequential(
    nn.Conv2D(in_channels=3,out_channels=3,kernel_size=16,dilation=16),
    nn.MaxPool2D(kernel_size=2),
    nn.Conv2D(in_channels=3,out_channels=3,kernel_size=8,dilation=20),
    nn.MaxPool2D(kernel_size=2)
)
net(x)

Tensor(shape=[1, 3, 230, 230], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[[[ 0.02829955,  0.26504278,  0.34046286, ...,  0.18637556,
            0.25362349,  0.23590879],
          [ 0.33989400,  0.42944476,  0.44870630, ...,  0.02599808,
            0.09757093,  0.10159671],
          [ 0.35378364,  0.45616937,  0.46879297, ...,  0.15715325,
            0.23273148,  0.24442036],
          ...,
          [ 0.11454862,  0.19976753,  0.19888361, ...,  0.20726545,
           -0.05703647,  0.16521169],
          [ 0.12638935,  0.21072203,  0.22215778, ...,  0.22123551,
            0.12364778,  0.20805614],
          [ 0.11984136,  0.21985191,  0.22385632, ...,  0.24592051,
            0.13062683,  0.21163845]],

         [[ 0.29594240,  0.10659514,  0.27000716, ...,  0.17051220,
            0.25503895,  0.15882725],
          [ 0.32607907,  0.07031970,  0.23571582, ...,  0.11310064,
            0.10874305,  0.03076977],
          [-0.04395428, -0.27822971, -0.06946186

Tensor(shape=[1, 3, 216, 216], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[[[ 0.52388060,  0.49636671,  0.31147841, ...,  0.14597557,
            0.31685925,  0.33904284],
          [ 0.50118214,  0.47941798,  0.43844092, ...,  0.28431973,
            0.26907071,  0.26923162],
          [ 0.40418479,  0.40939000,  0.40297547, ...,  0.25322801,
            0.31050646,  0.33872122],
          ...,
          [ 0.41765088,  0.38559318,  0.25195798, ...,  0.25590014,
            0.37370342,  0.40749723],
          [ 0.40564543,  0.37645158,  0.23736918, ...,  0.10963542,
            0.22380604,  0.18963815],
          [ 0.17306852,  0.20213334,  0.17506185, ...,  0.25290993,
            0.39110443,  0.39355919]],

         [[ 0.69141877,  0.53489488,  0.67873877, ...,  0.87960625,
            0.62592542,  0.90139687],
          [ 0.69914001,  0.83366764,  0.86651331, ...,  0.90645015,
            0.63892633,  0.87173194],
          [ 0.61306691,  0.81541860,  0.85591447

In [10]:
nn.Conv2D(in_channels=3,out_channels=3,kernel_size=3,stride=1,dilation=1)(x)

Tensor(shape=[1, 3, 1442, 1442], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[[[ 0.26680312,  0.01573536,  0.20800278, ..., -0.56695825,
           -0.60228074, -0.47792801],
          [-0.47617981, -0.55508018, -0.59719205, ..., -0.03088352,
           -0.15887265,  0.06514048],
          [ 0.04335696, -0.04263156, -0.17791314, ...,  0.01498421,
            0.21670209,  0.26701468],
          ...,
          [ 0.35161555,  0.11799452,  0.06803682, ..., -0.06342217,
           -0.00570719,  0.03283247],
          [ 0.02838009, -0.05607034,  0.00212690, ..., -0.39091155,
           -0.57765317, -0.29453683],
          [-0.29309753, -0.37691221, -0.55584103, ...,  0.11324559,
            0.06824206,  0.33860365]],

         [[ 0.27058211,  0.28463191,  0.58687556, ...,  0.45387435,
           -0.09074787, -0.07658187],
          [-0.06214085,  0.45454404, -0.08286504, ..., -0.01529661,
            0.16298696,  0.42660749],
          [ 0.40753984, -0.02143195,  0.165816

In [12]:
nn.Conv2D(in_channels=3,out_channels=3,kernel_size=3,stride=1,dilation=2)(x)

Tensor(shape=[1, 3, 1440, 1440], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[[[ 0.17052594,  0.16510679,  0.02625872, ..., -0.16758631,
           -0.32542911, -0.32703629],
          [-0.32641059, -0.32168686, -0.16269776, ...,  0.06898279,
            0.03409892, -0.17349271],
          [ 0.03651999, -0.17974246,  0.06433061, ...,  0.02256280,
            0.15727483,  0.14972794],
          ...,
          [ 0.05884962,  0.01333570, -0.25130218, ..., -0.29770663,
           -0.19107895, -0.14319462],
          [-0.19329557, -0.13917215, -0.28098679, ...,  0.23474272,
            0.00059780, -0.00585257],
          [ 0.00401054, -0.02329011,  0.22268559, ..., -0.25704280,
            0.06468223,  0.03002241]],

         [[-0.59727359, -0.49514958, -0.39913911, ...,  0.06160912,
            0.27218810,  0.22109075],
          [ 0.27291557,  0.22004046,  0.05734618, ..., -0.20886976,
           -0.35792214, -0.13150516],
          [-0.35153833, -0.12289782, -0.206351

In [13]:
nn.Conv2D(in_channels=3,out_channels=3,kernel_size=3,stride=2,dilation=2)(x)

Tensor(shape=[1, 3, 720, 720], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[[[-0.00638865,  0.25870931,  0.14589956, ..., -0.38974127,
           -0.24767429, -0.47745052],
          [-0.45199144, -0.44042966, -0.80095184, ...,  0.26209128,
            0.14995664, -0.01654933],
          [-0.48701766, -0.40577224, -0.25131980, ..., -0.44461277,
           -0.82078123, -0.45161986],
          ...,
          [-0.07681058, -0.02848649, -0.02677508, ..., -0.29319853,
           -0.60381252, -0.29728138],
          [-0.61557716, -0.56723362, -0.81240118, ..., -0.04191839,
           -0.01393600, -0.07028876],
          [-0.30325720, -0.27197209, -0.59079909, ..., -0.57483286,
           -0.81685978, -0.63135564]],

         [[ 0.16701178, -0.23728657, -0.06953931, ...,  0.16808949,
            0.02024814, -0.20109823],
          [ 0.16690877,  0.04255759,  0.28146479, ..., -0.23717026,
           -0.07315700,  0.16427819],
          [-0.20771231,  0.17335171,  0.02213908

In [14]:
nn.Conv2D(in_channels=3,out_channels=3,kernel_size=3,stride=2,dilation=5)(x)

Tensor(shape=[1, 3, 717, 717], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[[[ 0.21706928,  0.30962983,  0.43428421, ...,  0.35543793,
            0.00818520,  0.30553940],
          [ 0.19328313,  0.39971533,  0.35223234, ...,  0.30462849,
            0.42958409,  0.20479384],
          [ 0.31921399,  0.36960965,  0.00383302, ...,  0.40873289,
            0.35329732,  0.18590024],
          ...,
          [ 0.08630209,  0.06938826,  0.24411415, ...,  0.14532186,
            0.01501822,  0.31499666],
          [-0.22871938,  0.00017029,  0.00403140, ...,  0.08107496,
            0.25878289,  0.08481777],
          [ 0.29190594,  0.13959275, -0.00123333, ...,  0.00001552,
           -0.00824045, -0.22136334]],

         [[ 0.52504474,  0.10821185,  0.64550817, ...,  0.09134750,
           -0.13043495,  0.04772231],
          [ 0.30937871,  0.30854547,  0.21168879, ...,  0.10340994,
            0.64736181,  0.51219767],
          [ 0.02745091,  0.08498891, -0.12345767